<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 11</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(5)  # creating pseudo-random numbers for reproducibility

In [12]:
# Let's consider the following A_orig matrix:

A_orig = np.array([[4, 2, 4, 5, 4, 2, 2], [4, 4, 3, 2, 2, 5, 4], [4, 1, 4, 7, 3, 6, 2], [8 ,1, 2, 0, 5, 0, 7], [4 , 5, 8, 7, 6, 2, 3]],dtype=float)
M, N = 5, 7
print (pd.DataFrame(A_orig).head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  1.0  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  3.0


In [13]:
# Let's add some NaN(s) to  A_orig matrix

A = A_orig.copy()
A[3][1] = np.NaN
A[4][6] = np.NaN

A_df = pd.DataFrame(A)
print (A_df.head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  NaN  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  NaN


In [14]:
# Fine tune K value such that the (average percentage) error for the 2 missing elements is minimum.
# Remember: this is an exercise where we have the actual values for missing elements. 
# In real-life scenarios missins elements are not known and you may need different metric(s)
# to assess the quality of your reconstructed matrix.
K = 10
P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
P = np.divide(P, K*P.max())
Q = np.divide(Q, K*Q.max())

In [15]:
def matrix_factorization(Rating_Matrix, P, Q, K, steps, alpha=0.001, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    eij = Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eRating_Matrix = np.dot(P,Q)
        e = 0
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    e = e + pow(Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # print("Total error at step", step, "is", e)
        if e < 0.0001:
            break
    return P, Q.T

In [6]:
eP, eQ = matrix_factorization(A, P, Q.T, K, steps = 1000)
eA = np.matmul(eP, eQ.T)

print(A,'\n')
print(eA)

Total error at step 0 is 634.6094450116353
Total error at step 1 is 633.8705338359546
Total error at step 2 is 633.068259445681
Total error at step 3 is 632.1967651534414
Total error at step 4 is 631.2497294362087
Total error at step 5 is 630.2203322284658
Total error at step 6 is 629.1012202987864
Total error at step 7 is 627.8844719715468
Total error at step 8 is 626.5615615378399
Total error at step 9 is 625.1233237966849
Total error at step 10 is 623.559919281371
Total error at step 11 is 621.8608008578016
Total error at step 12 is 620.0146825337845
Total error at step 13 is 618.0095114916841
Total error at step 14 is 615.832444552802
Total error at step 15 is 613.4698305006075
Total error at step 16 is 610.9071999310373
Total error at step 17 is 608.129264559708
Total error at step 18 is 605.1199281946818
Total error at step 19 is 601.8623118736313
Total error at step 20 is 598.3387959576783
Total error at step 21 is 594.531082258826
Total error at step 22 is 590.4202795381743
Tot

In [8]:
nan_Indices = np.argwhere(np.isnan(A))
total_Error = 0

for index in nan_Indices:
    i, j = index
    total_Error += abs(eA[i, j] - A_orig[i, j]) / A_orig[i, j]

# Average percentage error
ape = total_Error / len(nan_Indices)

print("Average percentage error : ", ape)

Average percentage error :  2.5706370046692637


In [16]:
error = {}

for k in range(1,11):
    eP, eQ = matrix_factorization(A, P, Q.T, k, steps = 1000)
    eA = np.matmul(eP, eQ.T)

    nan_Indices = np.argwhere(np.isnan(A))
    total_Error = 0

    for index in nan_Indices:
        i, j = index
        total_Error += abs(eA[i, j] - A_orig[i, j]) / A_orig[i, j]
    # Average percentage error
    ape = total_Error / len(nan_Indices)
    error[k] = ape

    print("K :",k, "->", "Average percentage error : ", ape)

error


K : 1 -> Average percentage error :  1.3885016333003932
K : 2 -> Average percentage error :  4.411398534753277
K : 3 -> Average percentage error :  5.174666155053438
K : 4 -> Average percentage error :  5.19557058488917
K : 5 -> Average percentage error :  3.8123468520205344
K : 6 -> Average percentage error :  3.1589009078746377
K : 7 -> Average percentage error :  2.820114637837735
K : 8 -> Average percentage error :  2.610503313068751
K : 9 -> Average percentage error :  2.4642505455766335
K : 10 -> Average percentage error :  2.3554791957402004


{1: 1.3885016333003932,
 2: 4.411398534753277,
 3: 5.174666155053438,
 4: 5.19557058488917,
 5: 3.8123468520205344,
 6: 3.1589009078746377,
 7: 2.820114637837735,
 8: 2.610503313068751,
 9: 2.4642505455766335,
 10: 2.3554791957402004}

In [17]:
dict(
        filter(
            lambda item: item[1]==min(error.values()), 
            error.items()
            )
)

{1: 1.3885016333003932}